<a href="https://colab.research.google.com/github/sr606/LLM/blob/main/mermaid_trail_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#genearte_business_lineage
import json
from graphviz import Digraph


with open("business_lineage.json", "r") as f:
    model = json.load(f)


dot = Digraph("Business_Lineage", format="pdf")
dot.attr(rankdir="LR", fontsize="11")
dot.attr("node", shape="box", style="rounded,filled")


# Color scheme
COLORS = {
    "source": "#E6F2FF",
    "dimension": "#FFF2CC",
    "fact": "#E6FFE6",
    "exception": "#FDEDEC"
}


# -----------------------------
# Source Node
# -----------------------------
source = model["sources"][0]["table"]
dot.node("source", source, fillcolor=COLORS["source"])


previous_node = "source"


# -----------------------------
# Dimension Nodes
# -----------------------------
for i, dim in enumerate(model.get("dimensions", [])):
    node_id = f"dim_{i}"

    label = f"""{dim["table"]}\n
{dim["join_type"]}\n
ON {dim["condition"]}"""

    dot.node(node_id, label, fillcolor=COLORS["dimension"])

    dot.edge(previous_node, node_id)
    previous_node = node_id


# -----------------------------
# Fact Target
# -----------------------------
fact = model["fact_target"]["table"]
dot.node("fact", fact, fillcolor=COLORS["fact"])

dot.edge(previous_node, "fact")


# -----------------------------
# Exception Branch
# -----------------------------
if model.get("exception_output"):
    exc = model["exception_output"]
    dot.node("exception",
             f"{exc['name']}\nTrigger: {exc['trigger']}",
             fillcolor=COLORS["exception"])

    dot.edge("fact", "exception")


dot.render("business_lineage", view=True)

print("✅ business_lineage.pdf generated")


In [ ]:
BUSINESS_LINEAGE_PROMPT = """
You are a senior data architect.

Extract BUSINESS-LEVEL lineage only.

Ignore:
- Technical hash stages
- Internal dataset names
- Lookup helper tables
- Intermediate technical objects

Return JSON with:

{
  "pipeline_name": "",
  "sources": [],
  "dimensions": [],
  "fact_target": {},
  "exception_output": {},
  "business_rules": []
}

Only include:
- Primary source tables
- Major business dimension joins
- Join type
- Join condition
- Final fact target
- Exception outputs
- Important transformation rules

Return valid JSON only.
"""


In [ ]:
import json
import html
from graphviz import Digraph


# =====================================================
# 1️⃣ Load Pipeline Model
# =====================================================

with open("etl_pipeline_model.json", "r", encoding="utf-8") as f:
    model = json.load(f)


# =====================================================
# 2️⃣ Safe Text Utility
# =====================================================

def safe(text):
    if text is None:
        return ""
    return html.escape(str(text))


def format_section(title, lines):
    if not lines:
        return ""

    section = f"<TR><TD ALIGN='LEFT'><B>{safe(title)}</B></TD></TR>"
    for line in lines:
        section += f"<TR><TD ALIGN='LEFT'>• {safe(line)}</TD></TR>"
    return section


# =====================================================
# 3️⃣ Structured Formatters
# =====================================================

def format_input(inp):
    lines = []

    if isinstance(inp, dict):
        if inp.get("name"):
            lines.append(f"Stage: {inp['name']}")
        if inp.get("type"):
            lines.append(f"Type: {inp['type']}")
        if inp.get("query"):
            lines.append("Query: SQL Extract")
    else:
        lines.append(str(inp))

    return lines


def format_process(proc):
    lines = []

    if isinstance(proc, dict):
        if proc.get("name"):
            lines.append(f"Stage: {proc['name']}")
        if proc.get("type"):
            lines.append(f"Type: {proc['type']}")

        if proc.get("transformations"):
            for t in proc["transformations"][:2]:
                if isinstance(t, dict):
                    lines.append(f"Output: {t.get('output','')}")
    else:
        lines.append(str(proc))

    return lines


def format_join(join):
    lines = []

    if isinstance(join, dict):
        if join.get("type"):
            lines.append(f"Join Type: {join['type']}")

        keys = join.get("keys", [])
        for k in keys[:4]:
            lines.append(f"ON {k}")
    else:
        lines.append(str(join))

    return lines


def format_analytics(ana):
    lines = []

    if isinstance(ana, dict):
        if ana.get("group_by"):
            lines.append("Group By:")
            for g in ana["group_by"]:
                lines.append(g)

        if ana.get("metrics"):
            lines.append("Metrics:")
            for m in ana["metrics"]:
                lines.append(m)
    else:
        lines.append(str(ana))

    return lines


def format_output(out):
    lines = []

    if isinstance(out, dict):
        if out.get("name"):
            lines.append(f"Target: {out['name']}")
        if out.get("type"):
            lines.append(f"Type: {out['type']}")
        if out.get("link_file"):
            lines.append("Mode: File Output")
    else:
        lines.append(str(out))

    return lines


# =====================================================
# 4️⃣ Create Graphviz Graph
# =====================================================

dot = Digraph("ETL_Technical_Architecture", format="pdf")
dot.attr(rankdir="LR", fontsize="10")
dot.attr("node", shape="plaintext")


# =====================================================
# 5️⃣ Build Pipelines
# =====================================================

for p_index, pipeline in enumerate(model.get("pipelines", [])):

    with dot.subgraph(name=f"cluster_{p_index}") as c:
        c.attr(label=safe(pipeline.get("name", f"Pipeline_{p_index}")),
               fontsize="12",
               style="rounded")

        previous_node = None

        def create_node(node_id, header, color, sections):
            label = f"""
            <
            <TABLE BORDER="1" CELLBORDER="0" CELLSPACING="0" CELLPADDING="6">
                <TR>
                    <TD BGCOLOR="{color}"><B>{safe(header)}</B></TD>
                </TR>
                {sections}
            </TABLE>
            >
            """

            c.node(node_id, label=label)
            return node_id

        # --------------------
        # INPUTS
        # --------------------
        for i, inp in enumerate(pipeline.get("inputs", [])):
            node_id = f"{p_index}_IN_{i}"
            sections = format_section("Details", format_input(inp))
            current = create_node(node_id, "INPUT", "#E6F2FF", sections)

            if previous_node:
                c.edge(previous_node, current)
            previous_node = current

        # --------------------
        # PROCESSES
        # --------------------
        for i, proc in enumerate(pipeline.get("processes", [])):
            node_id = f"{p_index}_PROC_{i}"
            sections = format_section("Logic", format_process(proc))
            current = create_node(node_id, "PROCESS", "#E6FFE6", sections)

            if previous_node:
                c.edge(previous_node, current)
            previous_node = current

        # --------------------
        # JOINS
        # --------------------
        for i, join in enumerate(pipeline.get("joins", [])):
            node_id = f"{p_index}_JOIN_{i}"
            sections = format_section("Join Info", format_join(join))
            current = create_node(node_id, "JOIN", "#FFF2CC", sections)

            if previous_node:
                c.edge(previous_node, current)
            previous_node = current

        # --------------------
        # ANALYTICS
        # --------------------
        for i, ana in enumerate(pipeline.get("analytics", [])):
            node_id = f"{p_index}_ANA_{i}"
            sections = format_section("Analytics", format_analytics(ana))
            current = create_node(node_id, "ANALYTICS", "#F4E6FF", sections)

            if previous_node:
                c.edge(previous_node, current)
            previous_node = current

        # --------------------
        # OUTPUTS
        # --------------------
        for i, out in enumerate(pipeline.get("outputs", [])):
            node_id = f"{p_index}_OUT_{i}"
            sections = format_section("Target", format_output(out))
            current = create_node(node_id, "OUTPUT", "#E6FFFF", sections)

            if previous_node:
                c.edge(previous_node, current)
            previous_node = current


# =====================================================
# 6️⃣ Render PDF
# =====================================================

dot.render("etl_technical_architecture", view=True)

print("✅ Graphviz PDF generated → etl_technical_architecture.pdf")


In [ ]:
!pip install graphviz

In [ ]:
import re
import json
from graphviz import Digraph
from collections import defaultdict, deque


# =====================================================
# 1️⃣ CLASSIFY STAGE TYPE
# =====================================================

def classify_layer(stage_type):
    st = stage_type.upper()

    if "TRANSFORMER" in st:
        return "TRANSFORM"

    if "HASHED" in st:
        return "HASH"

    if "SEQ" in st:
        return "FILE"

    if "ORACLE" in st or "CUSTOM" in st:
        return "DB"

    return "OTHER"


# =====================================================
# 2️⃣ PARSE PSEUDOCODE INTO METADATA
# =====================================================

def parse_pseudocode(text):

    stages = {}
    dataset_producers = {}

    stage_pattern = r"--- \[(.*?) : (.*?)\]"
    input_pattern = r"Input:\s*←\s*(dataset_\d+)"
    output_pattern = r"Output:\s*→\s*(dataset_\d+)"
    stagevar_pattern = r"StageVar (.*)"
    constraint_pattern = r"Constraint \((.*?)\): (.*)"
    table_pattern = r"\bFROM\s+([\w\.]+)|\bJOIN\s+([\w\.]+)"

    current_stage = None
    collecting_sql = False

    for line in text.splitlines():

        stage_match = re.search(stage_pattern, line)
        if stage_match:
            stage_type = stage_match.group(1).strip()
            stage_name = stage_match.group(2).strip()

            current_stage = stage_name
            collecting_sql = False

            stages[current_stage] = {
                "type": stage_type,
                "layer": classify_layer(stage_type),
                "inputs": [],
                "outputs": [],
                "tables": [],
                "stagevars": [],
                "constraints": [],
                "joins": 0,
                "logic_complexity": 0
            }
            continue

        if current_stage:

            # Detect SQL start
            if "SQL:" in line:
                collecting_sql = True
                continue

            # Collect SQL lines
            if collecting_sql:
                tables = re.findall(table_pattern, line, re.IGNORECASE)
                for t in tables:
                    for table in t:
                        if table:
                            stages[current_stage]["tables"].append(table)

                if "JOIN" in line.upper():
                    stages[current_stage]["joins"] += 1

                if "CASE" in line.upper() or "IF" in line.upper():
                    stages[current_stage]["logic_complexity"] += 1

            # Inputs
            input_match = re.search(input_pattern, line)
            if input_match:
                stages[current_stage]["inputs"].append(input_match.group(1))

            # Outputs
            output_match = re.search(output_pattern, line)
            if output_match:
                dataset = output_match.group(1)
                stages[current_stage]["outputs"].append(dataset)
                dataset_producers[dataset] = current_stage

            # Stage variables
            stagevar_match = re.search(stagevar_pattern, line)
            if stagevar_match:
                stages[current_stage]["stagevars"].append(stagevar_match.group(1))

            # Constraints
            constraint_match = re.search(constraint_pattern, line)
            if constraint_match:
                stages[current_stage]["constraints"].append(constraint_match.group(2))

    return stages, dataset_producers


# =====================================================
# 3️⃣ COMPUTE EXECUTION DEPTH
# =====================================================

def compute_depths(stages, dataset_producers):

    graph = defaultdict(list)
    indegree = defaultdict(int)

    for stage, info in stages.items():
        for dataset in info["inputs"]:
            if dataset in dataset_producers:
                parent = dataset_producers[dataset]
                graph[parent].append(stage)
                indegree[stage] += 1

    depth = {}
    queue = deque()

    for stage in stages:
        if indegree[stage] == 0:
            queue.append(stage)
            depth[stage] = 0

    while queue:
        node = queue.popleft()
        for neighbor in graph[node]:
            indegree[neighbor] -= 1
            if indegree[neighbor] == 0:
                depth[neighbor] = depth[node] + 1
                queue.append(neighbor)

    return depth


# =====================================================
# 4️⃣ BUILD GRAPHVIZ FROM METADATA
# =====================================================

def build_graphviz(stages, dataset_producers):

    dot = Digraph("ETL_Flow", engine="dot")
    dot.attr(rankdir="LR", splines="spline", nodesep="0.8", ranksep="1.2")

    dot.attr("node",
             shape="box",
             style="rounded,filled",
             width="3",
             height="1")

    depths = compute_depths(stages, dataset_producers)

    # Detect final targets
    outgoing = defaultdict(int)
    for stage, info in stages.items():
        for dataset in info["inputs"]:
            if dataset in dataset_producers:
                parent = dataset_producers[dataset]
                outgoing[parent] += 1

    targets = [s for s in stages if outgoing[s] == 0]

    max_depth = max(depths.values())
    for t in targets:
        if stages[t]["layer"] == "DB":
            depths[t] = max_depth + 1

    # Group by depth
    levels = defaultdict(list)
    for stage, d in depths.items():
        levels[d].append(stage)

    # Create nodes
    for d in sorted(levels):
        with dot.subgraph() as s:
            s.attr(rank="same")
            for stage in levels[d]:
                info = stages[stage]

                label = f"{stage}\n"
                label += f"Type: {info['type']}\n"
                label += f"In: {len(info['inputs'])} | Out: {len(info['outputs'])}\n"

                if info["tables"]:
                    label += f"Tables: {len(set(info['tables']))}\n"

                if info["joins"] > 0:
                    label += f"Joins: {info['joins']}\n"

                if info["stagevars"]:
                    label += f"StageVars: {len(info['stagevars'])}\n"

                if info["constraints"]:
                    label += f"Constraints: {len(info['constraints'])}"

                color = "#F4F6F7"
                if info["layer"] == "DB":
                    color = "#AED6F1"
                elif info["layer"] == "TRANSFORM":
                    color = "#F9E79F"
                elif info["layer"] == "HASH":
                    color = "#ABEBC6"
                elif info["layer"] == "FILE":
                    color = "#F5B7B1"

                s.node(stage, label, fillcolor=color)

    # Add edges
    for stage, info in stages.items():
        for dataset in info["inputs"]:
            if dataset in dataset_producers:
                parent = dataset_producers[dataset]

                if stages[parent]["layer"] == "HASH":
                    dot.edge(parent, stage,
                             constraint="false",
                             color="gray")
                else:
                    dot.edge(parent, stage)

    return dot


# =====================================================
# 5️⃣ RUN + EXPORT
# =====================================================

with open("Samle_Job2 1 2_detailed_pseudocode.txt", "r") as f:
    pseudo_text = f.read()

stages, producers = parse_pseudocode(pseudo_text)

# Export JSON metadata
metadata = {
    "stages": stages,
    "dependencies": producers
}

with open("etl_metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)

# Build diagram
graph = build_graphviz(stages, producers)
graph.render("etl_flow_detailed", format="pdf")

print("✅ JSON exported as etl_metadata.json")
print("✅ Diagram generated as etl_flow_detailed.pdf")


✅ JSON exported as etl_metadata.json
✅ Diagram generated as etl_flow_detailed.pdf


In [ ]:
import re
import json
from collections import defaultdict


# ==========================================================
# 1️⃣ CLASSIFY STAGE LAYER
# ==========================================================

def classify_layer(stage_type):
    st = stage_type.upper()

    if "TRANSFORMER" in st:
        return "Transform"

    if "HASHED" in st:
        return "Intermediate"

    if "SEQ" in st:
        return "Outputs"

    if "ORACLE" in st or "CUSTOM" in st:
        return "Source_Target"

    return "Other"


# ==========================================================
# 2️⃣ PARSE PSEUDOCODE
# ==========================================================

def parse_pseudocode(text):

    stages = {}
    dataset_producers = {}

    stage_pattern = r"--- \[(.*?) : (.*?)\]"
    input_pattern = r"Input:\s*←\s*(dataset_\d+)\s*\((.*?)\)"
    output_pattern = r"Output:\s*→\s*(dataset_\d+)\s*\((.*?)\)"
    stagevar_pattern = r"StageVar (.*)"
    constraint_pattern = r"Constraint \((.*?)\): (.*)"
    table_pattern = r"\bFROM\s+([\w\.]+)|\bJOIN\s+([\w\.]+)"

    current_stage = None
    collecting_sql = False

    for line in text.splitlines():

        stage_match = re.search(stage_pattern, line)
        if stage_match:
            stage_type = stage_match.group(1).strip()
            stage_name = stage_match.group(2).strip()

            current_stage = stage_name
            collecting_sql = False

            stages[current_stage] = {
                "type": stage_type,
                "layer": classify_layer(stage_type),
                "inputs": [],
                "outputs": [],
                "stagevars": [],
                "constraints": [],
                "tables": [],
                "joins": 0,
                "business_rules": []
            }
            continue

        if not current_stage:
            continue

        if "SQL:" in line:
            collecting_sql = True
            continue

        if collecting_sql:
            tables = re.findall(table_pattern, line, re.IGNORECASE)
            for t in tables:
                for table in t:
                    if table:
                        stages[current_stage]["tables"].append(table)

            if "JOIN" in line.upper():
                stages[current_stage]["joins"] += 1

        input_match = re.search(input_pattern, line)
        if input_match:
            dataset = input_match.group(1)
            dataset_name = input_match.group(2)
            stages[current_stage]["inputs"].append((dataset, dataset_name))

        output_match = re.search(output_pattern, line)
        if output_match:
            dataset = output_match.group(1)
            dataset_name = output_match.group(2)
            stages[current_stage]["outputs"].append((dataset, dataset_name))
            dataset_producers[dataset] = current_stage

        stagevar_match = re.search(stagevar_pattern, line)
        if stagevar_match:
            rule = stagevar_match.group(1).strip()
            stages[current_stage]["stagevars"].append(rule)
            stages[current_stage]["business_rules"].append(rule)

        constraint_match = re.search(constraint_pattern, line)
        if constraint_match:
            rule = constraint_match.group(2).strip()
            stages[current_stage]["constraints"].append(rule)
            stages[current_stage]["business_rules"].append(rule)

        if " IF " in line.upper():
            stages[current_stage]["business_rules"].append(line.strip())

    return stages, dataset_producers


# ==========================================================
# 3️⃣ GENERATE MERMAID CODE
# ==========================================================

def generate_mermaid(stages, producers):

    lines = []
    lines.append("flowchart LR")

    layers = defaultdict(list)
    for stage, info in stages.items():
        layers[info["layer"]].append(stage)

    for layer, stage_list in layers.items():
        lines.append(f"  subgraph {layer}")
        for stage in stage_list:
            info = stages[stage]

            label = f"{info['type']}: {stage}"
            if info["joins"] > 0:
                label += f"<br/>Joins: {info['joins']}"
            if info["stagevars"]:
                label += f"<br/>StageVars: {len(info['stagevars'])}"
            if info["constraints"]:
                label += f"<br/>Constraints: {len(info['constraints'])}"

            lines.append(f'    {stage}["{label}"]')
        lines.append("  end\n")

    for stage, info in stages.items():
        for dataset, dataset_name in info["inputs"]:
            parent = producers.get(dataset)
            if parent:
                lines.append(
                    f'  {parent} -- "{dataset}: {dataset_name}" --> {stage}'
                )

    return "\n".join(lines)


# ==========================================================
# 4️⃣ EXPORT TO PDF-READY HTML
# ==========================================================

with open("Sample_Job1 1 2_detailed_pseudocode.txt", "r") as f:
    pseudo_text = f.read()

stages, producers = parse_pseudocode(pseudo_text)
mermaid_code = generate_mermaid(stages, producers)

html_template = f"""
<!DOCTYPE html>
<html>
<head>
  <script src="https://cdn.jsdelivr.net/npm/mermaid/dist/mermaid.min.js"></script>
  <script>mermaid.initialize({{ startOnLoad: true }});</script>
</head>
<body>
<div class="mermaid">
{mermaid_code}
</div>
</body>
</html>
"""

with open("etl_flow.html", "w") as f:
    f.write(html_template)

print("HTML generated → etl_flow.html")


✅ HTML generated → etl_flow.html
👉 Open in browser → Print → Save as PDF


In [ ]:
import re
import json
import os
from collections import defaultdict
from dotenv import load_dotenv
from openai import OpenAI

# ==========================================================
# 🔐 LOAD API KEY
# ==========================================================

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ==========================================================
# 1️⃣ CLASSIFY LAYER
# ==========================================================

def classify_layer(stage_type):
    st = stage_type.upper()

    if "TRANSFORMER" in st:
        return "Transform"

    if "HASHED" in st:
        return "Intermediate"

    if "SEQ" in st:
        return "Outputs"

    if "ORACLE" in st or "CUSTOM" in st:
        return "Source_Target"

    return "Other"


# ==========================================================
# 2️⃣ DETERMINISTIC PARSER
# ==========================================================

def parse_pseudocode(text):

    stages = {}
    dataset_producers = {}

    stage_pattern = r"--- \[(.*?) : (.*?)\]"
    input_pattern = r"Input:\s*←\s*(dataset_\d+)\s*\((.*?)\)"
    output_pattern = r"Output:\s*→\s*(dataset_\d+)\s*\((.*?)\)"
    stagevar_pattern = r"StageVar (.*)"
    constraint_pattern = r"Constraint \((.*?)\): (.*)"
    table_pattern = r"\bFROM\s+([\w\.]+)|\bJOIN\s+([\w\.]+)"

    current_stage = None
    collecting_sql = False

    for line in text.splitlines():

        stage_match = re.search(stage_pattern, line)
        if stage_match:
            stage_type = stage_match.group(1).strip()
            stage_name = stage_match.group(2).strip()

            current_stage = stage_name
            collecting_sql = False

            stages[current_stage] = {
                "type": stage_type,
                "layer": classify_layer(stage_type),
                "inputs": [],
                "outputs": [],
                "stagevars": [],
                "constraints": [],
                "tables": [],
                "joins": 0
            }
            continue

        if not current_stage:
            continue

        if "SQL:" in line:
            collecting_sql = True
            continue

        if collecting_sql:
            tables = re.findall(table_pattern, line, re.IGNORECASE)
            for t in tables:
                for table in t:
                    if table:
                        stages[current_stage]["tables"].append(table)

            if "JOIN" in line.upper():
                stages[current_stage]["joins"] += 1

        input_match = re.search(input_pattern, line)
        if input_match:
            dataset = input_match.group(1)
            dataset_name = input_match.group(2)
            stages[current_stage]["inputs"].append({
                "dataset_id": dataset,
                "dataset_name": dataset_name
            })

        output_match = re.search(output_pattern, line)
        if output_match:
            dataset = output_match.group(1)
            dataset_name = output_match.group(2)
            stages[current_stage]["outputs"].append({
                "dataset_id": dataset,
                "dataset_name": dataset_name
            })
            dataset_producers[dataset] = current_stage

        stagevar_match = re.search(stagevar_pattern, line)
        if stagevar_match:
            stages[current_stage]["stagevars"].append(stagevar_match.group(1).strip())

        constraint_match = re.search(constraint_pattern, line)
        if constraint_match:
            stages[current_stage]["constraints"].append(constraint_match.group(2).strip())

    return stages, dataset_producers


# ==========================================================
# 3️⃣ LLM ENRICHMENT
# ==========================================================

def enrich_stage_with_llm(stage_name, stage_metadata):

    prompt = f"""
You are an expert ETL architecture analyst.

Analyze the following stage metadata and return JSON only with:

- business_summary
- stage_role (fact_load, dimension_load, enrichment, staging, file_output, lookup)
- exception_logic
- lookup_logic
- data_enrichment_purpose

Metadata:
{json.dumps(stage_metadata, indent=2)}
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a precise ETL analyzer."},
            {"role": "user", "content": prompt}
        ]
    )

    content = response.choices[0].message.content

    try:
        return json.loads(content)
    except:
        return {"business_summary": content}


def enrich_all_stages(metadata):

    for stage, info in metadata["stages"].items():
        print(f"🔍 Enriching {stage} ...")
        enriched = enrich_stage_with_llm(stage, info)
        metadata["stages"][stage]["semantic"] = enriched

    return metadata


# ==========================================================
# 4️⃣ GENERATE MERMAID
# ==========================================================

def generate_mermaid(metadata):

    stages = metadata["stages"]
    producers = metadata["dependencies"]

    lines = []
    lines.append("flowchart LR\n")

    layers = defaultdict(list)
    for stage, info in stages.items():
        layers[info["layer"]].append(stage)

    for layer, stage_list in layers.items():
        lines.append(f"  subgraph {layer}")
        for stage in stage_list:
            info = stages[stage]

            label = f"{info['type']}: {stage}"

            if "semantic" in info:
                summary = info["semantic"].get("business_summary", "")
                summary = summary.replace('"', "'")
                label += f"<br/><i>{summary}</i>"

            lines.append(f'    {stage}["{label}"]')
        lines.append("  end\n")

    for stage, info in stages.items():
        for inp in info["inputs"]:
            dataset_id = inp["dataset_id"]
            dataset_name = inp["dataset_name"]
            parent = producers.get(dataset_id)

            if parent:
                lines.append(
                    f'  {parent} -- "{dataset_id}: {dataset_name}" --> {stage}'
                )

    return "\n".join(lines)


# ==========================================================
# 5️⃣ MAIN EXECUTION
# ==========================================================

with open("Sample_job1 1 2_detailed_pseudocode.txt", "r") as f:
    pseudo_text = f.read()

stages, producers = parse_pseudocode(pseudo_text)

metadata = {
    "stages": stages,
    "dependencies": producers
}

# 🔬 LLM enrichment (research phase)
metadata = enrich_all_stages(metadata)

# Save enriched JSON
with open("etl_metadata_enriched.json", "w") as f:
    json.dump(metadata, f, indent=4)

# Generate Mermaid
mermaid_code = generate_mermaid(metadata)

# Wrap in HTML for PDF
html = f"""
<!DOCTYPE html>
<html>
<head>
<script src="https://cdn.jsdelivr.net/npm/mermaid/dist/mermaid.min.js"></script>
<script>mermaid.initialize({{startOnLoad:true}});</script>
</head>
<body>
<div class="mermaid">
{mermaid_code}
</div>
</body>
</html>
"""

with open("etl_flow_ai.html", "w") as f:
    f.write(html)

print("✅ Enriched metadata → etl_metadata_enriched.json")
print("✅ Mermaid HTML → etl_flow_ai.html")
print("👉 Open HTML in browser → Print → Save as PDF")


In [ ]:
#analyze_job.py
import os
import json
import re
from dotenv import load_dotenv
from openai import AzureOpenAI

# Load env
load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")


def safe_parse(content):
    content = content.strip()

    # remove markdown
    content = re.sub(r"```json", "", content)
    content = re.sub(r"```", "", content)

    try:
        return json.loads(content)
    except:
        match = re.search(r"\{.*\}", content, re.DOTALL)
        if match:
            return json.loads(match.group())
        raise Exception("JSON Parse Failed")


def analyze_full_job(text):

    prompt = f"""
You are an ETL reverse engineering expert.

Analyze this entire job definition and extract logical pipeline architecture.

Return STRICT JSON:

{{
  "job_name": "",
  "pipelines": [
    {{
      "name": "",
      "inputs": [],
      "processes": [],
      "joins": [],
      "analytics": [],
      "outputs": []
    }}
  ]
}}

Rules:
- Group stages logically.
- Detect exception branches.
- Extract join types and join keys.
- Extract derived column logic.
- Extract output targets.
- Keep it structured and concise.
- Do not hallucinate.
"""

    response = client.chat.completions.create(
        model=DEPLOYMENT,
        temperature=0,
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": "Return JSON only."},
            {"role": "user", "content": prompt + "\n\nJOB:\n" + text}
        ]
    )

    return safe_parse(response.choices[0].message.content)


if __name__ == "__main__":

    with open("Sample_job.txt", "r", encoding="utf-8") as f:
        job_text = f.read()

    result = analyze_full_job(job_text)

    with open("etl_pipeline_model.json", "w", encoding="utf-8") as f:
        json.dump(result, f, indent=4)

    print("✅ Pipeline model saved → etl_pipeline_model.json")


In [ ]:
#render_diagram

import json

with open("etl_pipeline_model.json", "r", encoding="utf-8") as f:
    model = json.load(f)


def build_mermaid(model):

    lines = []
    lines.append("flowchart LR\n")

    # Color classes
    lines.append("classDef input fill:#e6f2ff;")
    lines.append("classDef process fill:#e6ffe6;")
    lines.append("classDef join fill:#fff2cc;")
    lines.append("classDef analytics fill:#f4e6ff;")
    lines.append("classDef output fill:#e6ffff;\n")

    for idx, pipeline in enumerate(model["pipelines"]):

        pname = f"PIPE_{idx}"
        lines.append(f"subgraph {pname}[{pipeline['name']}]")

        prev_node = None

        # INPUT
        for i, inp in enumerate(pipeline["inputs"]):
            node = f"{pname}_IN_{i}"
            lines.append(f'{node}["INPUT<br/>{inp}"]:::input')
            if prev_node:
                lines.append(f"{prev_node} --> {node}")
            prev_node = node

        # PROCESS
        for i, proc in enumerate(pipeline["processes"]):
            node = f"{pname}_PROC_{i}"
            lines.append(f'{node}["PROCESS<br/>{proc}"]:::process')
            if prev_node:
                lines.append(f"{prev_node} --> {node}")
            prev_node = node

        # JOIN
        for i, join in enumerate(pipeline["joins"]):
            node = f"{pname}_JOIN_{i}"
            lines.append(f'{node}["JOIN<br/>{join}"]:::join')
            if prev_node:
                lines.append(f"{prev_node} --> {node}")
            prev_node = node

        # ANALYTICS
        for i, ana in enumerate(pipeline["analytics"]):
            node = f"{pname}_ANA_{i}"
            lines.append(f'{node}["ANALYTICS<br/>{ana}"]:::analytics')
            if prev_node:
                lines.append(f"{prev_node} --> {node}")
            prev_node = node

        # OUTPUT
        for i, out in enumerate(pipeline["outputs"]):
            node = f"{pname}_OUT_{i}"
            lines.append(f'{node}["OUTPUT<br/>{out}"]:::output')
            if prev_node:
                lines.append(f"{prev_node} --> {node}")
            prev_node = node

        lines.append("end\n")

    return "\n".join(lines)


mermaid_code = build_mermaid(model)

html = f"""
<!DOCTYPE html>
<html>
<head>
<script src="https://cdn.jsdelivr.net/npm/mermaid/dist/mermaid.min.js"></script>
<script>
mermaid.initialize({{ startOnLoad: true, theme: "default" }});
</script>
</head>
<body>

<h2>ETL Pipeline Architecture</h2>

<div class="mermaid">
{mermaid_code}
</div>

</body>
</html>
"""

with open("etl_pipeline_diagram.html", "w", encoding="utf-8") as f:
    f.write(html)

print("✅ Diagram generated → etl_pipeline_diagram.html")
